# **problems that arose during code execution of Building and Deploying the RAG System on Google Colab**

During the execution of the vector store indexing with FAISS when you answer "yes" for using offline mode

In [ ]:
from langchain.vectorstores import FAISS

def create_vector_store(embeddings, chunks):
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    return vector_store

At first the requirements needs to be satisfied then you would have to upload a file from your computer. Of course you have the option to cancel the upload. Then comes the part where you have to type the CR document, here if you have already uploaded the document you type "/content/" + the naming of your doc. otherwise you can also copy the path of your document if you have it in your drive ( there is a part where you link the notebook to your drive in the code ).Then you ask your question about the content of the document while stating whether you'd use offline mode or not.
Now the issue comes afterwards:

The error AttributeError: 'list' object has no attribute 'embed_documents' is occurring in the create_vector_store function when called within the chatbot_interface function. This is because, in offline mode, the create_offline_embeddings function returns a list of embeddings. However, FAISS.from_texts expects an embedding function as the embedding argument, not a list of embeddings. This function is supposed to take a single text and return its embedding.


**Suggested Changes**

In [ ]:

from langchain.vectorstores import FAISS

def create_vector_store(embeddings, chunks):
  if isinstance( embdeddings, list):
     #Check if embeddings is a list (offline mode)
     vector_store= FAISS.from_embeddings(text_embeddings=zip(chunks, embeddings), embeddings=None)
     # Pass chunks and embeddings together
     # no need for an embeddings function
  else: # Online mode ( embeddings is an OpenAIEmebddings object)
       vector_store=FAISS.from_texts(chunks, embedding=embeddings)
  return vector_store


then another error surges


In [ ]:
NameError: name 'embdeddings' is not defined

The problem could be that even with the typo fixed, the isinstance condition might not be evaluating correctly, leading to unexpected code flow and potentially to a state where embeddings isn't actually assigned. This can happen if the return type of create_offline_embeddings and create_online_embeddings isn't what the isinstance check expects.

Here's a more detailed breakdown:

Return Type Mismatch: create_online_embeddings returns an OpenAIEmbeddings instance, which isn't a list. This means the if isinstance(embeddings, list) will evaluate to False when in online mode. This in itself isn't necessarily an error.
FAISS.from_texts Failure: The issue might arise if FAISS.from_texts requires a different structure for the embedding argument than an OpenAIEmbeddings object. If it expects a list or numpy array but gets an OpenAIEmbeddings object, that could cause the code to fail in online mode, even without a visible error message in your traceback.
OpenAIEmbeddings in Offline Mode: The create_offline_embeddings returns a list of embeddings, which is correct for offline mode. But if, somehow, create_online_embeddings is called in offline mode, then you'd get an OpenAIEmbeddings object passed to create_vector_store, causing it to enter the first branch of the if statement. That wouldn't match with the way offline mode is expected to work.

**Suggested Changes**

In [ ]:
def create_vector_store(embeddings, chunks):
  if isinstance( embdeddings, list):# Corrected typo: embeddings instead of embeddings
     #Check if embeddings is a list (offline mode)
     # In Offline mode, 'embeddings' is already a list of embeddings.
     vector_store= FAISS.from_embeddings(text_embeddings=zip(chunks, embeddings), embeddings= None)
  else: # Online mode ( embeddings is an OpenAIEmebddings object)
      #Online mode: First, compute embdeddings from text chunks
      embeddings_list= embeddings.embed_document(chunks) # call embed_document here
      vector_store=FAISS.from_embeddings(text_embeddings=zip(chunks, embeddings_list), embedding=None)
       # Pass embeddings to FAISS
  return vector_store


And yet the same type of error appears regardless of potential fixes

During the execution of the code if you type "no" for using offline mode:

In [ ]:
TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

The error TypeError: Client.__init__() got an unexpected keyword argument 'proxies' is happening because the httpx library version you are using (likely 0.28.0 or later) no longer supports the proxies keyword argument in the Client initialization.

The OpenAIEmbeddings class from langchain is likely initializing an OpenAI client under the hood, which in turn is trying to create an HTTP client (using httpx) with the proxies argument. This argument has been removed in newer versions of httpx, leading to the error.

**Suggested Changes**

Downgrade httpx: The issue is likely due to an incompatibility between the openai library and a recent version of httpx. By installing a compatible version (0.27.7 in this case), you're ensuring that the proxies keyword argument is still recognized.
Restart the runtime: After installing the specific httpx version, you might need to restart your runtime to ensure that the changes are applied. You can do this in Google Colab by going to "Runtime" -> "Restart runtime".

In [ ]:
!pip install httpx==0.27.7